In [1]:
from pathlib import Path
from pyspark.sql import SparkSession, DataFrame
from pyspark.ml.regression import LinearRegression, RandomForestRegressionModel
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler


BASE_DATA_DIR = Path().home() / "Documents/PySparkCurso/download"

spark: SparkSession = (
    SparkSession.builder.master("local").appName("Ml with spark").getOrCreate()
)

24/04/15 10:47:18 WARN Utils: Your hostname, IdeaPad-Gaming-3-15IHU6 resolves to a loopback address: 127.0.1.1; using 192.168.1.5 instead (on interface wlp0s20f3)
24/04/15 10:47:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/15 10:47:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
cars_temp: DataFrame = spark.read.csv(
    str(BASE_DATA_DIR / "Carros.csv"), inferSchema=True, header=True, sep=";"
)

cars_temp.show(5)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



In [4]:
cars: DataFrame = cars_temp.select("Consumo", "Cilindros", "Cilindradas", "HP")
cars.show(5)

+-------+---------+-----------+---+
|Consumo|Cilindros|Cilindradas| HP|
+-------+---------+-----------+---+
|     21|        6|        160|110|
|     21|        6|        160|110|
|    228|        4|        108| 93|
|    214|        6|        258|110|
|    187|        8|        360|175|
+-------+---------+-----------+---+
only showing top 5 rows



In [5]:
vec_carac = VectorAssembler(
    inputCols=[("Consumo"), ("Cilindros"), ("Cilindradas")],
    outputCol="characteristics",
)

In [6]:
cars = vec_carac.transform(cars)
cars.show(5)

+-------+---------+-----------+---+-----------------+
|Consumo|Cilindros|Cilindradas| HP|  characteristics|
+-------+---------+-----------+---+-----------------+
|     21|        6|        160|110| [21.0,6.0,160.0]|
|     21|        6|        160|110| [21.0,6.0,160.0]|
|    228|        4|        108| 93|[228.0,4.0,108.0]|
|    214|        6|        258|110|[214.0,6.0,258.0]|
|    187|        8|        360|175|[187.0,8.0,360.0]|
+-------+---------+-----------+---+-----------------+
only showing top 5 rows



In [7]:
cars_traine, car_test = cars.randomSplit([0.7, 0.3])

print(cars_traine.count(), car_test.count())

22 10
